In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
#install latest library
import os

os.chdir('/content/drive/MyDrive/voxelmorph_swin_unet_new')

In [2]:
# pip install patchify

In [3]:
import cremi.Dataset
from cremi.Dataset import mask
from cremi.Dataset import load_data
import numpy as np

### **Load Data**

In [ ]:
"""""""""
generate the binary mask
"""""""""

# def mask_binary(mask): 
#   m_l=[]
#   for i in range(len(mask)):
   
#     mask_binary = mask[i] !=100
#     # print(mask_binary)
#     mask_b = mask_binary.astype(np.int)
#     m_l.append(mask_b)
#   m_s = np.stack(m_l, axis = 0)
#   return m_s

In [5]:
# patched_fixed,patched_moving,patched_label_fixed,patched_label_moving = load_data() # using for the first time generation mask

### **Generate Binary Mask**

In [ ]:
""""""""""""""""
# using for the first time generation mask
""""""""""""""""

# print('generate mask...')
# mask_fixed, mask_moving = mask(patched_fixed, patched_moving, patched_label_fixed,patched_label_moving)  

# print('load mask...')
# mask_fixed_load = np.load('mask_fixed.npy')
# mask_moving_load = np.load('mask_moving.npy')

# print('generate binary mask...')
# mask_binary_fixed = mask_binary(mask_fixed_load)
# mask_binary_moving = mask_binary(mask_moving_load)
  
# print('save binary mask...')
# np.save('binary_fixed.npy',mask_binary_fixed)
# np.save('binary_moving.npy', mask_binary_moving)

In [ ]:
# print(mask_binary_fixed.shape, mask_binary_moving.shape)

In [4]:
b_f = np.load('binary_fixed.npy')
b_m = np.load('binary_moving.npy')

In [5]:
# np.unique(b_f)
# np.unique(b_m)

array([0, 1])

### **Evaluation**

In [7]:
 # The evaluation Dice from original VoxelMorph paper
 
 def dice(array1, array2, labels=None, include_zero=False):
    """
    Computes the dice overlap between two arrays for a given set of integer labels.
    Parameters:
        array1: Input array 1.
        array2: Input array 2.
        labels: List of labels to compute dice on. If None, all labels will be used.
        include_zero: Include label 0 in label list. Default is False.
    """
    if labels is None:
        labels = np.concatenate([np.unique(a) for a in [array1, array2]])
        labels = np.sort(np.unique(labels))
    if not include_zero:
        labels = np.delete(labels, np.argwhere(labels == 0)) 

    dicem = np.zeros(len(labels)) 
    for idx, label in enumerate(labels):
        top = 2 * np.sum(np.logical_and(array1 == label, array2 == label))
 
        bottom = np.sum(array1 == label) + np.sum(array2 == label)
        bottom = np.maximum(bottom, np.finfo(float).eps)  # add epsilon
 
        dicem[idx] = top / bottom
    return dicem

###########但是现在network用来直接比较registered和fixed的结果是不对的，得用segmentation的image去比较，放在network里了。

In [8]:
import torch

from torch import nn
from torch.nn import functional as F


def mydice(pred, true):
  pred = torch.from_numpy(pred) 
  true = torch.from_numpy(true) 

  intersection = (pred * true).sum()                            
  dice = (2.*intersection + 1)/(pred.sum() + true.sum() + 1)  

  return dice

In [ ]:
###2.generate mask moved through STN-->batch
mask_moved = STN(batch_binary_mask_moving,phi) ### phi is from network output

In [ ]:
##3.dice对比这2个数组
dice_score = dice(moved_mask, batch_mask_fixed)